<a href="https://colab.research.google.com/github/SurajitBiswasPhysics/surajit-biswas.github.io/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-13 16:07:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  85.5MB/s    in 0.3s    

2024-12-13 16:07:10 (85.5 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [55]:
# Count the number of ratings per user and per book
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Filter users with less than 200 ratings and books with less than 100 ratings
filtered_ratings = df_ratings[
    df_ratings['user'].isin(user_counts[user_counts >= 200].index) &
    df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)
]

# Merge book data with ratings to get book titles
df_ratings_merged = pd.merge(filtered_ratings, df_books, on='isbn', how='left')

# Create a pivot table: rows = users, columns = books, values = ratings
book_user_matrix = df_ratings_merged.pivot_table(index='user', columns='title', values='rating', fill_value=0)

# Ensure column order consistency
book_titles = book_user_matrix.columns.tolist()

# Convert the pivot table to a sparse matrix (for efficiency)
sparse_matrix = csr_matrix(book_user_matrix.values)


In [56]:
# Train KNN model
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(sparse_matrix.T)  # Transpose to ensure rows = books, columns = features (users)

NearestNeighbors(algorithm='brute', metric='cosine')

In [57]:
# Function to get recommendations
def get_recommends(book_title):
    if book_title not in book_titles:
        raise ValueError(f"'{book_title}' not found in book titles.")

    # Find the index of the input book title in the column list
    book_idx = book_titles.index(book_title)

    # Extract the feature vector for the book (1D array)
    book_vector = sparse_matrix.T[book_idx]  # Transpose again to get (1, n_features)

    # Get the nearest neighbors, including the book itself
    distances, indices = knn_model.kneighbors(book_vector, n_neighbors=6)

    # Collect recommendations (excluding the book itself)
    recommended_books = [
        [book_titles[indices.flatten()[i]], distances.flatten()[i]]
        for i in range(1, len(indices.flatten()))
    ]

    return [book_title, recommended_books]


In [69]:
df_ratings_merged.head()

,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
2,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
3,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
4,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff


In [60]:
books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(books)

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['Interview with the Vampire', 0.73450685], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Catch 22', 0.7939835]]]


In [70]:
books = get_recommends("The Poisonwood Bible: A Novel")
print(books)

['The Poisonwood Bible: A Novel', [['Jewel', 0.73224056], ['Midnight in the Garden of Good and Evil: A Savannah Story', 0.75103486], ['Empire Falls', 0.76218396], ['Gap Creek: The Story Of A Marriage', 0.7667542], ['Midwives: A Novel', 0.78507257]]]


In [62]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ['The Lovely Bones: A Novel',"I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.72,0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864], ['I Know This Much Is True', 0.7677075], ['The Surgeon', 0.7699411], ['The Weight of Water', 0.77085835], ["I'll Be Seeing You", 0.8016211]]]
You passed the challenge! 🎉🎉🎉🎉🎉
